In [63]:
# 🧠 Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib
import os


In [64]:
# 📂 Load and Explore Data
df = pd.read_csv("C:/Users/Administrator/Desktop/capstone/Civic-Assist/data/seed_data_cleaned.csv",header=None)
df.columns = ['text', 'department', 'category', 'severity']
df.head()





,text,department,category,severity
0,No light on 2nd Cross since last night,Electricity Board,Streetlight Outage,low
1,Streetlight near the park is not working,Electricity Board,Streetlight Outage,medium
2,Multiple streetlights fused on main road,Electricity Board,Streetlight Outage,medium
3,Dark stretch near bus stop due to broken stree...,Electricity Board,Streetlight Outage,high
4,Lamp post flickering opposite temple,Electricity Board,Streetlight Outage,low


In [65]:
# 🔍 Check for Nulls and Class Distribution
df.info()
df['department'].value_counts()
df['category'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        87 non-null     object
 1   department  87 non-null     object
 2   category    87 non-null     object
 3   severity    87 non-null     object
dtypes: object(4)
memory usage: 2.8+ KB


category
Streetlight Outage    19
Waste Collection      19
Road Maintenance      19
Manhole Hazard        15
Pipe Leakage          15
Name: count, dtype: int64

In [66]:
# 🧹 Preprocess Text (Basic)
df['clean_text'] = df['text'].str.lower()
df['text'] = (
    df['text']
    .str.lower()
    .str.replace(r'[^\w\s]', ' ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)




In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_text'])


In [68]:
from sklearn.model_selection import train_test_split

# Department
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, df['department'], test_size=0.2, random_state=42)

# Category
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, df['category'], test_size=0.2, random_state=42)

# Severity
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, df['severity'], test_size=0.2, random_state=42)


In [69]:
from sklearn.linear_model import LogisticRegression

# Department model
dept_model = LogisticRegression(max_iter=1000)
dept_model.fit(X_train_d, y_train_d)

# Category model
cat_model = LogisticRegression(max_iter=1000)
cat_model.fit(X_train_c, y_train_c)

# Severity model
severity_model = LogisticRegression(max_iter=1000)
severity_model.fit(X_train_s, y_train_s)



LogisticRegression(max_iter=1000)

In [70]:
from sklearn.metrics import classification_report

print("📌 Department Classification Report:")
print(classification_report(y_test_d, dept_model.predict(X_test_d)))

print("📌 Category Classification Report:")
print(classification_report(y_test_c, cat_model.predict(X_test_c)))

print("📌 Severity Classification Report:")
print(classification_report(y_test_s, severity_model.predict(X_test_s)))


📌 Department Classification Report:
                         precision    recall  f1-score   support

      Electricity Board       1.00      0.50      0.67         4
Public Works Department       0.50      0.50      0.50         2
 Solid Waste Management       1.00      1.00      1.00         5
Water Supply & Sewerage       0.78      1.00      0.88         7

               accuracy                           0.83        18
              macro avg       0.82      0.75      0.76        18
           weighted avg       0.86      0.83      0.82        18

📌 Category Classification Report:
                    precision    recall  f1-score   support

    Manhole Hazard       1.00      0.60      0.75         5
      Pipe Leakage       1.00      1.00      1.00         2
  Road Maintenance       0.40      1.00      0.57         2
Streetlight Outage       1.00      0.75      0.86         4
  Waste Collection       1.00      1.00      1.00         5

          accuracy                           

In [ ]:
import joblib
import os

os.makedirs("models", exist_ok=True)

joblib.dump(dept_model, "models/department_model.pkl")
joblib.dump(cat_model, "models/category_model.pkl")
joblib.dump(severity_model, "models/severity_model.pkl")
joblib.dump(vectorizer, "models/vectorizer.pkl")



['models/vectorizer.pkl']

In [72]:
from sklearn.metrics import accuracy_score
y_pred = dept_model.predict(X_test_d)
print("Department Model Accuracy:", accuracy_score(y_test_d, y_pred))
print(df['department'].value_counts())
print(df['category'].value_counts())
print(df['severity'].value_counts())




Department Model Accuracy: 0.8333333333333334
department
Water Supply & Sewerage    30
Electricity Board          19
Public Works Department    19
Solid Waste Management     19
Name: count, dtype: int64
category
Streetlight Outage    19
Waste Collection      19
Road Maintenance      19
Manhole Hazard        15
Pipe Leakage          15
Name: count, dtype: int64
severity
low       32
medium    28
high      27
Name: count, dtype: int64


In [73]:
from sklearn.metrics import classification_report

y_pred = dept_model.predict(X_test_d)
print(classification_report(y_test_d, y_pred))


                         precision    recall  f1-score   support

      Electricity Board       1.00      0.50      0.67         4
Public Works Department       0.50      0.50      0.50         2
 Solid Waste Management       1.00      1.00      1.00         5
Water Supply & Sewerage       0.78      1.00      0.88         7

               accuracy                           0.83        18
              macro avg       0.82      0.75      0.76        18
           weighted avg       0.86      0.83      0.82        18

